Author: Quan Nguyen

Add in navigation links (table of contents)

# Asynchronous Programming in Python

In this tutorial, we will learn about the idea behind asynchronous programming as well as its application in online data collection using the Python programming language. We will discuss the general structure of an asynchronous program in Python, how to convert a sequential, synchronous program to an asynchronour one, and finally apply what we learn to web scraping processes.

## Introduction to Asynchronous Programming

Asynchronous programming is model of programming that focuses on coordinating different tasks in an application with the goal that the application would use the least amount of time to finish executing those tasks. In essence, asynchronous programming is about switching from a task to another one in a program when it is appropriate to create overlapping between waiting and processing time, and from that shorten the total time taken to finish the whole program.

### Simultaneous Chess Analogy

To understand the idea behind asynchronous programming, let us consider a commonly used analogy:

You are the chess world champion, and you have been challenged to a game of simultaneous chess, in which you have to play ten different other chess players at the same time.

<table class="image">
    <caption align="bottom" style="text-align: center">Simultaneous Chess by Samuel Reshevsky</caption>
    <tr><td><img width=500 src="https://image.ibb.co/h2oizp/Samuel_Reshevsky_age_8_defeating_several_chess_masters_at_once_in_France_1920.jpg"></img></td></tr>
</table>

There are also a number of specifications to this simultaneous chess game:
- It takes you an average of 3 minutes to make a move in a single game.
- It takes each of your opponents an average of 5 minutes to make a move in a single game.
- It takes an avarage of 20 moves from each player for a single game to end (i.e. 40 moves in total).

A simple approach for you to finish all ten chess games with your opponents is to face each of them separately and sequentially. In other words, you take on the first opponent in a continous game, which we know will on average take:

`3 minutes for your move * 20 moves + 5 minutes for an opponent's move * 20 moves = 160 minutes`

So to go through all opponents sequentially, it will take:

`160 minutes * 10 opponents = 1,600 minutes ≈ 27 hours`

Now, it is quite intuitive to see that a better approach is to alternate between different opponents: instead of completely finishing a specific game in an uninterrupted manner, you can switch to a different opponent as soon as you have made a move in one particular game. Specifically, say after you make a move in the game with opponent 1, you will now switch to opponent 2 and make your move, and you will keep switching to the next opponent immediately after making your move until you make a move in the game with opponent 10, at which point you will come back to opponent 1.

While you move through from opponent 2 to opponent 10, opponent 1 has had sufficient time to think and make their next move, and when you come back to them, you simply can think and make your own move. It can be seen that by **overlapping** the time it took for opponent $n$ to make their move and the time it took for you to make your own move in the game with opponent $n + 1$, you can save a significant amount of time it would take in total to finish all ten games.

Specifically, by immediately switching to the next opponent after making a move, you will be able to completely bypass the time it would take waiting for the individual opponents to make their moves if you were to face them in continuous games. Let us call the whole process of switching from opponent 1, through all opponents, all the way to opponent 10 a _round_. Then the time it takes for you to finish a round would be:

`3 minutes for your move * 10 opponents = 30 minutes`

As mentioned, after you make your move in the game with opponent 10, enough time has passed for opponent 1 to make their move, and the same goes for opponent 2, 3, ..., 10 when you get to them. This means that in each game with a specific opponent, two moves are made after just one round. The totoal time it takes for all games to be completed is therefore:

`30 minutes * 20 moves = 600 minutes = 10 hours`

Comparing to the 27 hours it would take to do separate, continuous games, this approach seems to be much favorable. In fact, this is what simultaneous chess players do in practice.

### Asynchronous Programming in Practice

The act of switching between opponents in simultaneous chess to create overlapping between independent actions is in essence the idea behind asynchronous programming. In most computer applications, there is a mixture of both processing and waiting tasks in the execution of the application: processing is simply the instructions of analyzing and manipulating data, while waiting can be waiting for a file input to be read in, waiting for a server to send back a response for a request, etc. By switching the execution flow of our program from a task that is currently waiting to a new task, we can now process this new task while the old task is concurrently waiting.

In short, asynchronous programming is in essence coordinating the independent waiting and processing tasks in an application to create overlapping between them, thus improving the speed of the application.

For example, in our simultaneous chess analogy, waiting for a specific opponent to make their move can be considered as a waiting task, and thinking and making your own move in the game with another opponent is a processing task. These tasks are independent from each other (as they are for different games, different opponents) and can therefore be overlapped so that a shorter execution time can be achieved.

We will also be implementing an asynchronous web-scraping engine later in this article, which makes requests to download data from different websites. Since the tasks of waiting for the server of a website to response and processing the response from another website are independent from each other, this program will have a better speed than its corresponding sequential version.

Another advantage of asynchronous programming is improved responsiveness. Say you have in a program an ordered list of independent tasks to be executed, and the first task is a relatively long-running (heavy-weight) one. If the program was executed sequentially, it would first have to spend a significant amount of time to finish the first task (as it is a long-running task) before it moves to the other faster, lighter tasks.

In an asynchronous program, since the execution swtiches between independent waiting and processing tasks, the ones that have less running time will finish executing before the ones with longer running time, so even with a heavy-weight task as the first item on the task list, the program will still complete light-weight tasks with less running time first. This will consequently lead to better responsiveness for the program, as the users will have the results returned from the light-weight tasks in a timely manner even if they are far behind in the task list. We will analyze the improvement in responsiveness achieved with asynchronous programming in later sections.

## Asynchronous Programming in Python

In this section we will be discussing the specifics of implementing an asynchronous program in Python. First we will go over the general structure of an asynchronous program and its main elements, then we will learn about specific APIs that Python provides to facilitate asynchronous programming, and finally we will try our hands with a starting problem of programming asynchrously.

### Coroutines, Event Loops, and Futures

Coroutines, event loops, and futures are the essential elements to an asynchronous program:
- An event loop is the main coordinator of tasks in an asynchronous program. It keeps track of all the tasks that are to be run asynchronously and decides which of those should be executed at a given moment. In other words, the event loop handles the task-switching aspect (or the execution flow) of asynchronous programming.
- Coroutines are a special type of functions that wraps around specific tasks so that they can be executed asynchronously. A coroutine is required to specify where in the function the task-switching event should take place--this is when the execution flow is returned from the function back to the event loop. Coroutines are typically created by the event loop, and stored internally in a task queue.
- Futures  are placeholders for results returned from coroutines. These future objects are created as soon as coroutines are initiated by the event loop, so they can represent actual results, pending execution if the coroutines are still running, or even exceptions if that are what the coroutines will return.

An event loop, coroutines and their corresponding futures are the core elements to an asynchronous program. First, the event loop is started and creates a task queue. A coroutine for the first task is then executed and its corresponding future is created. When a task-switching event is to take place inside this coroutine, the coroutine is suspended and another coroutine will be called. If this coroutine is a blocking function (e.g. input/output processing, sleeping, etc.), the execution flow is released back to the event loop, which will then execute the next item in the task queue.

This process will repeat for all items of the task queue, and as the task-switching event takes place in the coroutine for the last task, the execution flow will be given to the coroutine of the first task again. During this process, as a task finishes executing, it will be eliminated from the task queue, its coroutine will be terminated, and the corresponding future will register the returned result from that coroutine. The process will go on until all tasks in the queue are completely executed or terminated.

The following diagram further illustrates the general structure of an asynchronous program described above:

<table class="image">
    <caption align="bottom" style="text-align: center">
        Asynchronous Programming Structure<br>
        <a href="https://medium.freecodecamp.org/a-guide-to-asynchronous-programming-in-python-with-asyncio-232e2afa44f6">Source</a>
    </caption>
    <tr><td><img width=500 src="https://image.ibb.co/iJ2PvU/0_s1_GH0_YO9_ZNd_EEDxo.jpg"></img></td></tr>
</table>

### Python API

With the general structure of asynchronous programs in mind, let us consider specific APIs that Python provides for their implementation. The first foundation for these APIs is the `async` and `await` keywords that were added in Python 3.5, which are used to specify different elements of coroutines.

Specifically, `async` is typically put in front of the `def` keyword when a function is declared. A function with the `async` keyword in front will be registered as a coroutine by the Python interpreter. Furthermore, as we have discussed, inside each coroutine there has to be a specification regarding when a task-switching event will take place; `await` is used to mark when and where in a coroutine the execution flow will be released. This is typically done by waiting for another coroutine (`await coroutine`) or helper functions from the `asyncio` module, which is the topic we discuss below.

`asyncio` is a built-in Python module that provides advanced functionalities to manage event loops of asynchronous programs. With `asyncio`, you can initiate and manipulate coroutines and event loops with intuitive and easy function calls without extensive boilerplate code. These functions include:
- `asyncio.get_event_loop()`: The method returns an event loop for the current context, which is an object from the `asyncio.AbstractEventLoop` class. Most of the time we do not need to worry about this class, as the `asyncio` module already provides high-level API manage the event loops.
- `AbstractEventLoop.create_task()`: The method is to be called by an event loop. It adds the parameter passed in to the current task queue of the calling event loop; the input should be a coroutine.
- `AbstractEventLoop.run_until_complete()`: The method is also to be called by an event loop. It takes in the main coroutine of an asynchronous program as its parameter and executes it until the corresponding future of the coroutine is returned. While the method initiates the event loop execution, it also blocks all subsequent code following it until all futures are complete.
- `AbstractEventLoop.run_forever()`: This method is somewhat similar to `run_until_complete()`, except that, as suggested by the method name, the calling event loop will run forever unless the `AbstractEventLoop.stop()` method is called. So instead of exiting, the loop will continue running even upon obtaining the returned futures.
- `AbstractEventLoop.stop()`: The method causes the calling event loop to stop executing and exit at the nearest appropriate opportunity without causing the whole program to crash.

Aside from these methods, we additionally use a number of non-blocking functions to facilitate the task switching event. These include:
- `asyncio.sleep()`: While in itself a coroutine, this function creates an additional coroutine that completes after a given time (specified by the input in seconds). It is typically used as `asyncio.sleep(0)` to cause an immediate task switching event.
- `asyncio.wait()`: This function is also a coroutine, hence can be used to switch tasks. It takes in a sequence (usually a list) of futures and waits for them to complete their execution.

## Starting Examples in Python

In this section, we will explore the API that we discussed above in specific examples. Through these examples you will be familiarized with the procedure of implementing an asynchronous program in Python as well as observe the advantages that asynchronous programming provides.

### Asynchronous Countdowns

The goal of this example is to illustrate the overlapping of processing and waiting time of independent tasks. To do this we will be analyzing the countdown function below:

In [1]:
import time

def seq_countdown(name, delay):
    # padding space for formatting
    indents = (ord(name) - ord('a')) * '\t'
    
    n = 3
    while n:
        time.sleep(delay)
        
        # elapsed time from the beginning
        duration = time.perf_counter() - start
        print('-' * 40)
        # message of the form: duration letter = countdown
        print(f'{duration:.4f} \t{indents}{name} = {n}')
        
        n -= 1

The `seq_countdown()` function takes in a letter string (`'a'`, `'b'`, '`'c'`, etc.) and a delay time. It will then count down from three to one in seconds while printing out the time elapsed from the beginning of the program and the input letter string (with the current countdown number). Let us now see this function in action:

In [2]:
start = time.perf_counter()

seq_countdown('a', 1)
seq_countdown('b', 0.8)
seq_countdown('c', 0.5)

print('-' * 40)
print('Finished.')

----------------------------------------
1.0053 	a = 3
----------------------------------------
2.0059 	a = 2
----------------------------------------
3.0083 	a = 1
----------------------------------------
3.8109 		b = 3
----------------------------------------
4.6133 		b = 2
----------------------------------------
5.4161 		b = 1
----------------------------------------
5.9173 			c = 3
----------------------------------------
6.4230 			c = 2
----------------------------------------
6.9287 			c = 1
----------------------------------------
Finished.


Here we are calling `seq_countdown()` with different string letters and different delay times. Again, the numbers at the beginning of the lines indicate total numbers of seconds elapsed from the beginning of the program. We see that the program counted down for letter "a" first with one-second intervals, and it moved on to letter bB" with 0.8-second intervals, and finally letter "c" with 0.5-second intervals. This is a purely sequential, synchronous program since there is no overlapping between processing and waiting time anywhere. Additionally, it took approximately 6.9 seconds to run the program, which is the sum of counting-down time of all three letters.

`1 second x 3 (for "a") + 0.8 seconds x 3 (for "b") + 0.5 seconds x 3 (for "c") = 6.9 seconds`

Keeping the idea behind asynchronous programming in mind, we see that it would actually be beneficial for us to convert this program to an asynchronous one. Specifically, let's say during the first second of the program while we are waiting to count down the letter "a", we can potentially switch tasks to move to other letters. In fact, we will implement this setup for all the letters inside the countdown function. In other words, we will turn the countdown function into a coroutine.

Theoretically, now that all counting down tasks are coroutines in an asynchronous program, we should achieve better execution time and responsiveness for our program. Since all three tasks are processed independently, the countdown messages should be printed out out of order (jumping between different letters), and asynchronous program should only take about the same time that the largest task takes (i.e. 3 seconds for letter "a").

But first, let us turn our program asynchronous. To do this we first need to make the countdown function into a coroutine and specify a point inside the function to be a task switching event. In other words, we will add the keyword `async` in front of the function, and instead of the `time.sleep()` function, we will be using `asyncio.sleep()` together with the `await` keyword; the rest of the function should remain the same. Our countdown coroutine should now be:

In [3]:
import asyncio

async def async_countdown(name, delay):
    # padding space for formatting
    indents = (ord(name) - ord('a')) * '\t'
    
    n = 3
    while n:
        await asyncio.sleep(delay)
        
        # elapsed time from the beginning
        duration = time.perf_counter() - start
        print('-' * 40)
        # message of the form: duration letter = countdown
        print(f'{duration:.4f} \t{indents}{name} = {n}')
        
        n -= 1

Keeping the main elements of an asynchronous program in mind, we now need to initialize and manage an event loop. Specifically, we create an empty event loop with the `asyncio.get_event_loop()` method, add all of the three countdown tasks into the task queue with `AbstractEventLoop.create_task()`, and finally start running the event loop with `AbstractEventLoop.run_until_complete()`:

In [4]:
loop = asyncio.get_event_loop() # creating the event loop
# adding tasks to the task queue
tasks = [
    loop.create_task(async_countdown('a', 1)),
    loop.create_task(async_countdown('b', 0.8)),
    loop.create_task(async_countdown('c', 0.5))
]

start = time.perf_counter()
# run the event loop until all tasks are complete
loop.run_until_complete(asyncio.wait(tasks))

print('-' * 40)
print('Done.')

----------------------------------------
0.5056 			c = 3
----------------------------------------
0.8056 		b = 3
----------------------------------------
1.0017 	a = 3
----------------------------------------
1.0083 			c = 2
----------------------------------------
1.5141 			c = 1
----------------------------------------
1.6112 		b = 2
----------------------------------------
2.0076 	a = 2
----------------------------------------
2.4168 		b = 1
----------------------------------------
3.0132 	a = 1
----------------------------------------
Done.


Now we can see how asynchronous program can improve execution time and responsiveness of our programs. Instead of executing individual tasks sequentially, our program now switches between different countdowns and overlaps their processing/waiting time. This, as we discussed, results in different letters being printed out in between each other or simultaneously.

At the beginning of the program, instead of waiting for the whole first second to print out the first message "a = 3", the program switches to the next task in the task queue, in this case it is waiting for 0.8 seconds for the letter "b". This process continues until 0.5 seconds have passed, and "c = 3" is printed out, and 0.3 seconds later (at time 0.8 seconds), "b = 3" is printed out. This all happens before "a = 3" is printed out.

This task-switching property of our asynchronous program makes it significantly more responsive. Instead of "hanging" for one second before the first message is printed, the program now only takes 0.5 seconds (the shortest waiting period) to print out its first message. As for execution time, we see that this time it only takes three seconds in total to execute the whole program (instead of 6.9 seconds). This corresponds to what we speculated--that the execution time would be right around the time it takes to execute the largest task.

### Asynchronous Fibonacci

Let us try applying asynchronous programming to another problem. In this example we will consider the task of calculating different numbers in the Fibonacci sequence. Our starting function will look like:

In [18]:
def seq_fib(n):
    # if n = 0, say 0, if n = 1, say 1
    if n in [0, 1]:
        print(f'{n}: {n}')
        print(f'Took {time.perf_counter() - start:.2f} seconds.')
    
    # sequentially calculating fib(n)
    a, b = 1, 2
    i = 1
    while i < n:
        a, b = b, a + b
        i += 1
    
    # printing the last 20 digits if the result is too large
    print(f'{n}: {a % (10 ** 20)}')
    # printing the time elapsed from the beginning
    print(f'Took {time.perf_counter() - start:.2f} seconds.')

Now we will use this function to generate different numbers in the sequence:

In [19]:
start = time.perf_counter()

seq_fib(1000000)
seq_fib(1000)
seq_fib(20)

1000000: 42277359244926937501
Took 12.33 seconds.
1000: 91902245245323403501
Took 12.33 seconds.
20: 10946
Took 12.33 seconds.


As discussed above, this is one of the situations in which the first task in the program is so large that the execution appears to be hanging while processing this task. This is undesirable in terms of application responsiveness. Furthermore, the other two tasks were light-weight enough that it took almost no time to execute them after the first task.

Now, if we were to implement asynchronous programming for this program, 